#PRELIMINARIES

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

#Preliminaries:
url_train = "https://github.com/lakigigar/Caltech-CS155-2021/blob/main/projects/project1/WILDFIRES_TRAIN.zip?raw=TRUE"
url_test = "https://github.com/lakigigar/Caltech-CS155-2021/blob/main/projects/project1/WILDFIRES_TEST.zip?raw=TRUE"
df_train = pd.read_csv(url_train, compression="zip", index_col="id")
df_test = pd.read_csv(url_test, compression = "zip", index_col="id")

In [ ]:
#Peek at data
df_train

,LATITUDE,LONGITUDE,STATE,DISCOVERY_TIME,FIRE_SIZE,FIPS_NAME,FIPS_CODE,SOURCE_REPORTING_UNIT_NAME,DATE,LABEL
id,,,,,,,,,,
0,38.205000,-120.335000,CA,130.0,0.10,NaN,NaN,Stanislaus National Forest,1992-01-01,1
1,33.813100,-85.104300,GA,1115.0,1.17,Haralson,143.0,Georgia Forestry Commission,1992-01-01,4
2,32.201000,-82.498700,GA,1600.0,0.07,Montgomery,209.0,Georgia Forestry Commission,1992-01-01,2
3,32.509300,-81.708600,GA,1215.0,4.40,Bulloch,31.0,Georgia Forestry Commission,1992-01-01,4
4,33.663889,-116.171944,CA,NaN,0.20,NaN,NaN,CDF - Riverside Unit,1992-01-01,2
5,33.166700,-116.634200,CA,1330.0,5.00,NaN,NaN,Southern California Agency,1992-01-01,2
6,31.275800,-83.755500,GA,1500.0,3.20,Colquitt,71.0,Georgia Forestry Commission,1992-01-01,3
7,34.532900,-85.008200,GA,1600.0,9.60,Gordon,129.0,Georgia Forestry Commission,1992-01-01,3
8,32.132500,-82.761000,GA,10.0,0.58,Wheeler,309.0,Georgia Forestry Commission,1992-01-01,4


#DATA: FORMATTING

In [ ]:
#Data has a lot of NaNs and three labels of categorical data, which we must first transform

#Let's build a dataframe of the input data we already have in usable form
input_data = df_train[['LATITUDE', "LONGITUDE","DISCOVERY_TIME","FIRE_SIZE","FIPS_CODE", "LABEL"]]

#This leaves us STATE, FIPS_NAME, FIPS_CODE, SOURCE_REPORTING_UNIT_NAME, and DATE to mess with

#Date column:
from datetime import datetime
day_of_year=df_train['DATE'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d').timetuple().tm_yday)
input_data["DAY_OF_YR"] = day_of_year

#State Column:


#FIPS_NAME Column:


#FIPS_CODE Column:


#SOURCE_REPORTING_UNIT_NAME Column:



#Peek at data now
input_data[0:10]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


,LATITUDE,LONGITUDE,DISCOVERY_TIME,FIRE_SIZE,FIPS_CODE,LABEL,DAY_OF_YR
id,,,,,,,
0,38.205000,-120.335000,130.0,0.10,NaN,1,1
1,33.813100,-85.104300,1115.0,1.17,143.0,4,1
2,32.201000,-82.498700,1600.0,0.07,209.0,2,1
3,32.509300,-81.708600,1215.0,4.40,31.0,4,1
4,33.663889,-116.171944,NaN,0.20,NaN,2,1
5,33.166700,-116.634200,1330.0,5.00,NaN,2,1
6,31.275800,-83.755500,1500.0,3.20,71.0,3,1
7,34.532900,-85.008200,1600.0,9.60,129.0,3,1
8,32.132500,-82.761000,10.0,0.58,309.0,4,1


# DATA: CLEANING

In [ ]:
#For now, let's just drop all the rows with NA values, but we can look at other options later
input_data = input_data.dropna()

#Peek at data again
input_data

,LATITUDE,LONGITUDE,DISCOVERY_TIME,FIRE_SIZE,FIPS_CODE,LABEL,DAY_OF_YR
id,,,,,,,
1,33.813100,-85.104300,1115.0,1.17,143.0,4,1
2,32.201000,-82.498700,1600.0,0.07,209.0,2,1
3,32.509300,-81.708600,1215.0,4.40,31.0,4,1
6,31.275800,-83.755500,1500.0,3.20,71.0,3,1
7,34.532900,-85.008200,1600.0,9.60,129.0,3,1
...,...,...,...,...,...,...,...
285327,41.098800,-120.772200,1654.0,0.10,35.0,2,349
285337,34.718333,-118.798889,2349.0,0.10,37.0,2,352
285356,34.188333,-118.021944,1634.0,0.10,37.0,2,359


#SET-UP FOR MODEL TRAINING

In [ ]:
#Now that we have usable data, we should split the resulting data into 4 sets for cross-validation, and separate by input/output
from sklearn.model_selection import KFold

nsplits = 4

kf = KFold(n_splits = nsplits, shuffle = True)

output = input_data['LABEL']
input = input_data.drop(columns = 'LABEL')

<generator object _BaseKFold.split at 0x7f2234b95db0>
TRAIN: [    0     1     2 ... 98446 98448 98449] TEST: [    3     4    14 ... 98450 98451 98452]
TRAIN: [    0     1     3 ... 98450 98451 98452] TEST: [    2     5     7 ... 98440 98441 98445]
TRAIN: [    2     3     4 ... 98450 98451 98452] TEST: [    0     1     6 ... 98443 98448 98449]
TRAIN: [    0     1     2 ... 98450 98451 98452] TEST: [   22    23    25 ... 98434 98439 98446]


,LATITUDE,LONGITUDE,DISCOVERY_TIME,FIRE_SIZE,FIPS_CODE,DAY_OF_YR
id,,,,,,
1,33.813100,-85.104300,1115.0,1.17,143.0,1
2,32.201000,-82.498700,1600.0,0.07,209.0,1
3,32.509300,-81.708600,1215.0,4.40,31.0,1
6,31.275800,-83.755500,1500.0,3.20,71.0,1
7,34.532900,-85.008200,1600.0,9.60,129.0,1
...,...,...,...,...,...,...
285327,41.098800,-120.772200,1654.0,0.10,35.0,349
285337,34.718333,-118.798889,2349.0,0.10,37.0,352
285356,34.188333,-118.021944,1634.0,0.10,37.0,359


#TRAIN THE MODEL

In [ ]:
from sklearn import tree

#Testing on a single decision tree for kicks
for train_index, test_index in kf.split(input_data):
     print("TRAIN:", train_index, "TEST:", test_index)
     X_train, X_test = input.iloc[train_index], input.iloc[test_index]
     y_train, y_test = output.iloc[train_index], output.iloc[test_index]

     clf = tree.DecisionTreeClassifier()
     clf.fit(X_train, y_train)
     predictedlabels = clf.predict(X_test)
     testlabels = y_test.to_numpy()
     percent_correct = np.sum(predictedlabels==testlabels)/len(predictedlabels)
     print("Fraction correctly labelled: ", percent_correct)


TRAIN: [    1     3     4 ... 98450 98451 98452] TEST: [    0     2     7 ... 98435 98439 98448]
0.5887706183472821
TRAIN: [    0     1     2 ... 98447 98448 98451] TEST: [   14    19    21 ... 98449 98450 98452]
0.5874944135213098
TRAIN: [    0     2     3 ... 98450 98451 98452] TEST: [    1     4     5 ... 98432 98438 98440]
0.5881444764961605
TRAIN: [    0     1     2 ... 98449 98450 98452] TEST: [    3    15    22 ... 98443 98447 98451]
0.5859505139560395
